# <center> <img src="img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: <Coloca el nombre de tu carrera aqui>** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 08**

**Fecha**: 04 abril 2025

**Nombre del Equipo**: Arriba Linux

**Integrantes**: Tirzah Peniche Barba / Ana Cristina Lina Arellano / Juan Pedro Bihouet Partida

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

### Creación de la Conexión con el Cluster de Spark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://ac7f0d7e8e91:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-01b639bc-91c0-458d-b3e8-1d2e181e7363;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

### Creación de Kafka Stream

In [3]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "78a305ddc318:9093") \
                .option("subscribe", "kafka-spark-example") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform Binary Data into String

In [4]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

### Configuración del "Sink" del Stream

In [5]:

query = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(30)
query.stop()

25/04/04 14:07:07 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4ceddb8a-9ec7-412d-a77d-d1abe00237fe. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/04 14:07:07 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/04 14:07:08 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-----+---------+------+---------+-------------+---------+
|key|value|topic|partition|offset|timestamp|timestampType|value_str|
+---+-----+-----+---------+------+---------+-------------+---------+
+---+-----+-----+---------+------+---------+-------------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+----------------------+-------------------+---------+------+-----------------------+-------------+---------+
|key |value                 |topic              |partition|offset|timestamp              |timestampType|value_str|
+----+----------------------+-------------------+---------+------+-----------------------+-------------+---------+
|NULL|[70 72 75 65 62 61 73]|kafka-spark-example|0        |4     |2025-04-04 14:07:12.975|0            |pruebas  |
+----+----------------------+-------------------+---------+------+-----------------------+-------------+---------+



25/04/04 14:07:18 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3525 milliseconds


-------------------------------------------
Batch: 2
-------------------------------------------
+----+----------------------------------------------+-------------------+---------+------+-----------------------+-------------+------------+
|key |value                                         |topic              |partition|offset|timestamp              |timestampType|value_str   |
+----+----------------------------------------------+-------------------+---------+------+-----------------------+-------------+------------+
|NULL|[4D EF BF BD C3 A1 73 20 70 72 75 65 62 61 73]|kafka-spark-example|0        |5     |2025-04-04 14:07:17.894|0            |M�ás pruebas|
+----+----------------------------------------------+-------------------+---------+------+-----------------------+-------------+------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+----+-------------------+-------------------+---------+------+-----------------------+-------------+---------+
|key |value              |topic              |partition|offset|timestamp              |timestampType|value_str|
+----+-------------------+-------------------+---------+------+-----------------------+-------------+---------+
|NULL|[4A 65 6A 65 6A 65]|kafka-spark-example|0        |6     |2025-04-04 14:07:20.949|0            |Jejeje   |
+----+-------------------+-------------------+---------+------+-----------------------+-------------+---------+



In [6]:
sc.stop()